In [1]:
import os

In [2]:
%pwd

'/workspaces/Chicken-Disease-Classification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/workspaces/Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    region: str
    bucket: str
    path: str
    local_data_file: Path
    unzip_dir: Path
    aws_access_key: str
    aws_secret_key: str

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.utils import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        secrets_filepath=SECRETS_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.secrets = read_yaml(secrets_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        secrets = self.secrets.s3

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            region=secrets.region,
            bucket=secrets.bucket,
            path=secrets.path,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
            aws_access_key=os.environ.get('AWS_ACCESS_KEY_ID'),
            aws_secret_key=os.environ.get('AWS_SECRET_ACCESS_KEY')
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import boto3
from src.cnnClassifier import logger
from src.cnnClassifier.utils.utils import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        self.s3_client = boto3.client('s3', region_name=self.config.region)
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading {self.config.path} from {self.config.bucket}...")
            self.s3_client.download_file(self.config.bucket, self.config.path, str(self.config.local_data_file))
            logger.info(f"Download complete: {self.config.local_data_file}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-07-20 21:00:48,995: INFO: utils: yaml file: config/config.yaml loaded successfully]
[2023-07-20 21:00:48,998: INFO: utils: yaml file: secrets.yaml loaded successfully]
[2023-07-20 21:00:49,000: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-07-20 21:00:49,002: INFO: utils: created directory at: artifacts]
[2023-07-20 21:00:49,003: INFO: utils: created directory at: artifacts/data_ingestion]
[2023-07-20 21:00:49,018: INFO: credentials: Found credentials in environment variables.]
[2023-07-20 21:00:49,130: INFO: 3553449986: Downloading chicken-fecal-images.zip from chicken-fecal-images...]
[2023-07-20 21:00:52,733: INFO: 3553449986: Download complete: artifacts/data_ingestion/data.zip]
